In [ ]:
import gym
import numpy as np
import tensorflow as tf
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def experiment(alpha = .99, gamma = 0.1, n_episodes = 10000, max_action = 100):
    env = gym.make('FrozenLake-v0')

    tf.reset_default_graph()
    tf.set_random_seed(73)

    states = tf.placeholder(shape=[1,16], dtype=tf.float32)
    weights = tf.Variable(tf.random_uniform([16,4],minval=0, maxval=0.01))
    output = tf.matmul(states, weights)
    action = tf.argmax(output, 1)

    target_output = tf.placeholder(shape=[1,4], dtype=tf.float32)
    loss = tf.reduce_sum(tf.square(target_output - output))
    trainer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
    update = trainer.minimize(loss)

    
    Qs = [] # Weights matrices
    Res = [] # Final results
    Scores = [] # Cumulative rewards (Scores)
    Steps = [] # Steps per episode
    
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        sess.run(init)
        for i in range(n_episodes):
            s = env.reset()
            acc_rew = 0
            for j in range(max_action):
                a = sess.run(action, feed_dict={states: np.identity(16)[s:s+1]})
                Q = sess.run(output, feed_dict={states: np.identity(16)[s:s+1]})
                s_new, reward, done, info = env.step(a[0])
                Qnext = sess.run(output, feed_dict={states: np.identity(16)[s_new:s_new+1]})
                amax = np.max(Qnext)
                reward -= 0.1
                Q[0,a[0]] = reward + gamma * amax
                sess.run(update, feed_dict={states: np.identity(16)[s:s+1],target_output:Q})
                if done == True:
                    Res.append(reward)
                    Steps.append(j)
                    acc_rew += reward
                    Scores.append(acc_rew)
                    break
                else:
                    s = s_new
                    acc_rew += reward
    env.close()
    return {"results": np.array(Res), "steps": np.array(Steps), "scores": np.array(Scores), "Q": weights}

In [ ]:
res = experiment()

In [ ]:
# %load "../mylibrary.py"
def ma(ts, q):
    acc = 0
    res = []
    for i in range(q, len(ts) - q):
        for j in range(i - q, i + q):
            acc += ts[j]
        res.append(acc / (2 * q + 1))
        acc = 0
    return res

In [ ]:
q = 48

import matplotlib.pyplot as plt
%matplotlib inline

# Scores
x = range(len(res["scores"])-2*q)
plt.figure(figsize=(15,5))
plt.plot(x, ma(res["scores"], q))

# Steps
x = range(len(res["scores"])-2*q)
plt.figure(figsize=(15,5))
plt.plot(x, ma(res["steps"],q))

# Steps
x = range(np.max(res["steps"]) + 1)
plt.figure(figsize=(15,5))
y = np.zeros(np.max(res["steps"]) + 1)
for i in range(len(res["steps"])):
    y[(res["steps"][i])] += 1 
plt.plot(x,y)

In [ ]:
np.mean(res["scores"])

In [ ]:
print(res["Q"])
print(np.argmax(res["Q"],axis=1))